## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# Prep for file reading from google drive
from google.colab import drive

drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [3]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'],inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
application_df[list(application_df.columns)].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_types = application_df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

## OPTIMIZATION 1: Use more bins for application types and classification types
* Changed cutoff for applicaation_types from 500 to 50
* Changed cutoff for classifications from 1500 to 30

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [i for i, value in application_types.items() if value < 50]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classifications = application_df['CLASSIFICATION'].value_counts()
classifications

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_above_one_df = application_df[application_df.groupby('CLASSIFICATION')['CLASSIFICATION'].transform('size')>1]
classifications_above_one_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [i for i, value in classifications.items() if value < 30]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Other      227
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_encoded_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
application_encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays

# Separate the y variable, the labels
y = application_encoded_df["IS_SUCCESSFUL"]

# Separate the X variable, the features
X = application_encoded_df.drop(columns="IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential(name="BaseModel")

# First hidden layer
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=60))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "BaseModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4880      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,341
Trainable params: 7,341
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
from keras.callbacks import ModelCheckpoint

# create a checkpoint
checkpoint = ModelCheckpoint("/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/model_weights.h5", monitor='accuracy', verbose=1, save_weights_only=True, save_freq='epoch', period=5)

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5799 - accuracy: 0.7120
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5495 - accuracy: 0.7311
Epoch 5/100
785/804 [============================>.] - ETA: 0s - loss: 0.5478 - accuracy: 0.7308
Epoch 5: saving model to /content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/model_weights.h5
804/804 [==============================] - 4s 4ms/step - loss: 0.5477 - accuracy: 0.7306
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5462 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5443 - accuracy: 0.7337
Epoch 9/100

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5605 - accuracy: 0.7306 - 511ms/epoch - 2ms/step
Loss: 0.5604737401008606, Accuracy: 0.7306122183799744


In [29]:
# Export our model to HDF5 file
nn_model.save("/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/AlphabetSoupCharityOpt1.h5")


In [30]:
!ls "/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/"

AlphabetSoupCharity.h5			alphabet_soup.ipynb
AlphabetSoupCharityOpt1.h5		model_weights.h5
AlphabetSoupCharity_Optimization.ipynb


## Optimization 1: Rebinning Evaluation
Increased accuracy slightly from 72.95% to 73.06%

## Optimization 2: Change Hidden Activation Layers From Sigmoid to Relu

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_opt2 = tf.keras.models.Sequential(name="OptLayerType")

# First hidden layer
# Add our first Dense layer, including the input layer
nn_model_opt2.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=60))

# Second hidden layer
nn_model_opt2.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_opt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_opt2.summary()

Model: "OptLayerType"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                4880      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,341
Trainable params: 7,341
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn_model_opt2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# create a checkpoint
checkpoint = ModelCheckpoint("/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/model_weights_opt2.h5", monitor='accuracy', verbose=1, save_weights_only=True, save_freq='epoch', period=5)

# Train the model
fit_model2 = nn_model_opt2.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7387
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5345 - accuracy: 0.7398
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5337 - accuracy: 0.7400
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7388
Epoch 5/100
799/804 [============================>.] - ETA: 0s - loss: 0.5336 - accuracy: 0.7398
Epoch 5: saving model to /content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/model_weights_opt2.h5
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7400
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5335 - accuracy: 0.7400
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5336 - accuracy: 0.7401
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7409
Epoch 

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_opt2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5647 - accuracy: 0.7334 - 470ms/epoch - 2ms/step
Loss: 0.5646519064903259, Accuracy: 0.7334110736846924


In [34]:
# Export our model to HDF5 file
nn_model_opt2.save("/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/AlphabetSoupCharityOpt2.h5")

## Optimization 2: Activation Type Change Evaluation
Accuracy increased very slightly from 73.06% to 73.34%

## Optimization 3: Increase Number of Hiddlen Layers and Neurons

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model_opt3 = tf.keras.models.Sequential(name="OptLayerNumber")

# First hidden layer
# Add our first Dense layer, including the input layer
nn_model_opt3.add(tf.keras.layers.Dense(units=500, activation="relu", input_dim=60))

# Second hidden layer
nn_model_opt3.add(tf.keras.layers.Dense(units=300, activation="relu"))

# Third hidden layer
nn_model_opt3.add(tf.keras.layers.Dense(units=150, activation="relu"))

# Output layer
nn_model_opt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_opt3.summary()

Model: "OptLayerNumber"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 500)               30500     
                                                                 
 dense_11 (Dense)            (None, 300)               150300    
                                                                 
 dense_12 (Dense)            (None, 150)               45150     
                                                                 
 dense_13 (Dense)            (None, 1)                 151       
                                                                 
Total params: 226,101
Trainable params: 226,101
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn_model_opt3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# create a checkpoint
checkpoint = ModelCheckpoint("/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/model_weights_opt3.h5", monitor='accuracy', verbose=1, save_weights_only=True, save_freq='epoch', period=5)

# Train the model
fit_model3 = nn_model_opt3.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 12s 13ms/step - loss: 0.5686 - accuracy: 0.7243
Epoch 2/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5533 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5502 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 11s 13ms/step - loss: 0.5471 - accuracy: 0.7331
Epoch 5/100
798/804 [============================>.] - ETA: 0s - loss: 0.5466 - accuracy: 0.7326
Epoch 5: saving model to /content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/model_weights_opt3.h5
804/804 [==============================] - 6s 7ms/step - loss: 0.5465 - accuracy: 0.7328
Epoch 6/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5448 - accuracy: 0.7344
Epoch 7/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5444 - accuracy: 0.7347
Epoch 8/100
804/804 [==============================] - 8s 11ms/step - loss: 0.5428 - accuracy: 0.7359

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_opt3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5901 - accuracy: 0.7308 - 1s/epoch - 5ms/step
Loss: 0.5900996923446655, Accuracy: 0.7308454513549805


In [39]:
# Export our model to HDF5 file
nn_model_opt3.save("/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/AlphabetSoupCharityOpt3.h5")

## Optimization 3: More Hidden Layers and Neurons
Accuracy decreased very slightly from 73.34% to 73.08%


In [40]:
!ls "/content/gdrive/MyDrive/Colab Notebooks/Alphabet Soup/"

AlphabetSoupCharity.h5			alphabet_soup.ipynb
AlphabetSoupCharityOpt1.h5		model_weights.h5
AlphabetSoupCharityOpt2.h5		model_weights_opt2.h5
AlphabetSoupCharityOpt3.h5		model_weights_opt3.h5
AlphabetSoupCharity_Optimization.ipynb
